## Mapping the tweet location with folium ##

In [1]:
import json
import pandas as pd
import collections
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**use the twitter nltk sample**

In [2]:
%%bash

ls /home/mick/nltk_data/corpora/twitter_samples/

negative_tweets.json
positive_tweets.json
README.md
README.txt
tweets.20150430-223406.json


In [3]:
df = pd.read_json('/home/mick/nltk_data/corpora/twitter_samples/tweets.20150430-223406.json', lines=True, orient='records')

In [4]:
df.head()

,in_reply_to_status_id,contributors,filter_level,coordinates,id_str,favorited,created_at,favorite_count,in_reply_to_status_id_str,retweeted_status,...,in_reply_to_user_id_str,lang,retweet_count,timestamp_ms,user,geo,possibly_sensitive,entities,source,extended_entities
0,NaN,NaN,low,None,593891099434983424,False,2015-04-30 21:34:06+00:00,0,NaN,"{'in_reply_to_status_id': None, 'contributors'...",...,NaN,en,0,2015-04-30 21:34:06.725,"{'statuses_count': 74617, 'id_str': '107794703...",None,False,"{'symbols': [], 'trends': [], 'user_mentions':...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN
1,NaN,NaN,low,None,593891099548094464,False,2015-04-30 21:34:06+00:00,0,NaN,NaN,...,NaN,en,0,2015-04-30 21:34:06.752,"{'statuses_count': 126843, 'id_str': '55742250...",None,False,"{'symbols': [], 'trends': [], 'user_mentions':...","<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",NaN
2,NaN,NaN,low,None,593891099388846080,False,2015-04-30 21:34:06+00:00,0,NaN,"{'in_reply_to_status_id': None, 'contributors'...",...,NaN,en,0,2015-04-30 21:34:06.714,"{'statuses_count': 4571, 'id_str': '3006692193...",None,False,"{'media': [{'type': 'photo', 'source_status_id...","<a href=""http://twitter.com/download/android"" ...","{'media': [{'type': 'photo', 'source_status_id..."
3,NaN,NaN,low,None,593891100429045760,False,2015-04-30 21:34:06+00:00,0,NaN,"{'in_reply_to_status_id': None, 'contributors'...",...,NaN,en,0,2015-04-30 21:34:06.962,"{'statuses_count': 7119, 'id_str': '455154030'...",None,False,"{'media': [{'type': 'photo', 'source_status_id...","<a href=""http://twitter.com/download/iphone"" r...","{'media': [{'type': 'photo', 'source_status_id..."
4,NaN,NaN,low,None,593891100768784384,False,2015-04-30 21:34:07+00:00,0,NaN,"{'in_reply_to_status_id': None, 'contributors'...",...,NaN,en,0,2015-04-30 21:34:07.043,"{'statuses_count': 3543, 'id_str': '187547338'...",None,False,"{'media': [{'type': 'photo', 'source_status_id...","<a href=""http://tapbots.com/tweetbot"" rel=""nof...","{'media': [{'type': 'photo', 'source_status_id..."


In [5]:
df = df.assign(dtime = pd.to_datetime(df.created_at))

**work with the source field of the tweets**

In [6]:
from bs4 import BeautifulSoup

In [7]:
sources_simplified = []
for line in list(df.source):
    soup = BeautifulSoup(line,'lxml')
    f = soup.findAll('a')
    
    sources_simplified.append(f[0].text)
    #break

In [8]:
cnt = collections.Counter(sources_simplified)

In [9]:
dfsource = pd.DataFrame.from_dict(cnt,orient='index')

how the tweets were tweeted ?

In [10]:
dfsource.head()

,0
Twitter Web Client,4632
dlvr.it,190
Twitter for Android,3421
Twitter for iPhone,5743
Tweetbot for iΟS,123


In [11]:
df = df.assign(source_s = sources_simplified)

**get the iphone users**

In [12]:
dfIphone = df[df.source_s == 'Twitter for iPhone']

extract locations from the tweets

In [13]:
locations =  []
for l in list(dfIphone['user']):
    #print(l['location'])
    locations.append(l['location'])

In [16]:
from geopy.geocoders import Nominatim,Bing
geolocator = Nominatim(user_agent="research tool")

BING_KEY = "ArbF7ISMus8rlGUh0x8olUwz7-C2nzb0TB3JwzKkcqaTP6Krl5T9V6NYAyuLAoGc" ## get the bing key from the bing developer website
geolocator = Bing(BING_KEY)

def getCoords(location):
    try:
        locx = geolocator.geocode(location)
        return locx.latitude, locx.longitude
    except:
        return 0., 0.


**use time.sleep with random interval to avoid floodin the service**

In [17]:
import time

In [18]:
import numpy as np

In [19]:
np.random.uniform(0.1,0.3)

0.24600488696494874

create popups for folium

In [20]:

popups = {}
idx = 0
for location in locations:
    if(location != ''):
        popups[location] = idx
    idx += 1
        

In [21]:
popus = df.iloc[list(popups.values())].text

In [23]:

coordDictionary = {}
idx = 0
for location in locations:
    if(location != ''):
        coords = getCoords(location)
        coordDictionary[location] = coords
        print(location, coords)
    time.sleep(np.random.uniform(0.1,0.3))
    
    if(idx > 500):
        break
        
    idx += 1
    

chesterfield (53.24285888671875, -1.4353300333023071)
Lewisham, London (51.45915985107422, -0.012129999697208405)
glasgow (55.85783004760742, -4.242509841918945)
Madrid (40.42028045654297, -3.7057700157165527)
London / Surrey (39.89912, -83.44545)
Liverpool (53.410091400146484, -2.9784300327301025)
London (51.50642013549805, -0.12721000611782074)
scotland (56.69036865234375, -4.044013977050781)
The Old School House (27.946666717529297, -82.46527862548828)
west of the wash/north of IoW (42.07469940185547, -93.49996948242188)
United Kingdom (53.9438362121582, -2.5505640506744385)
West Midlands. UK.  (52.470001220703125, -2.2899999618530273)
Camblesforth, North Yorkshire (53.72650909423828, -1.017496943473816)
London (51.50642013549805, -0.12721000611782074)
Woking and Scunthorpe (53.586490631103516, -0.6517999768257141)
Whitstable (51.362308502197266, 1.0266400575637817)
south-east London (51.4900016784668, -0.0697610005736351)
Upper Gwladys Optimist (0.0, 0.0)
The Land of Dragons!...Wal

In [24]:
import pickle
pickle.dump(coordDictionary, open("coords.pickle","wb"))

In [25]:
idx = 0
cnew = []
for c in coordDictionary.values():
    p = popus.iloc[idx]
    cnew.append(p)
    idx += 1
    

In [26]:
clist = []
for k, c in coordDictionary.items():
    if(c[0] != 0.0):
        #print(c)
        clist.append(c)

### Mapping iphone users ###

In [27]:
import folium

In [28]:


m = folium.Map(location=[55.5074, 0.1278],zoom_start=5)

i = 0

for c in clist:
    folium.Marker(location=[c[0],c[1]]).add_to(m)
    i += 1

In [29]:
m

**source study**

In [30]:
dfsource.columns = ['counts']
dfsource.sort_values(by='counts',ascending=False)

,counts
Twitter for iPhone,5743
Twitter Web Client,4632
Twitter for Android,3421
Twitter for iPad,2568
Twitter for Android Tablets,861
Twitter for Windows Phone,427
twitterfeed,382
TweetDeck,306
dlvr.it,190
Mobile Web (M5),172


### Exercises ###

* make the coordinate locator smarter, avoiding duplicates
